# Cambio de formato de texto a un número fijo de columnas
2.16

- Problema
        
        Tiene cadenas largas que desea reformatear para que llenen un número especificado por el usuario de columnas.  


- Solución
        
        Utilice el módulo de envoltura de texto para reformatear el texto para la salida. Por ejemplo, suponga que tiene la siguiente cadena larga:

In [20]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."

A continuación, le mostramos cómo puede usar el módulo de envoltura de texto para reformatearlo de varias maneras:

In [3]:
import textwrap

In [27]:
print(dir(textwrap))

['TextWrapper', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_leading_whitespace_re', '_whitespace', '_whitespace_only_re', 'dedent', 'fill', 'indent', 're', 'shorten', 'wrap']


In [4]:
print(textwrap.fill(s, 70))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.


In [5]:
print(textwrap.fill(s, 40))

Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.


In [7]:
print(textwrap.fill(s, 40, initial_indent="     "))

     Look into my eyes, look into my
eyes, the eyes, the eyes, the eyes, not
around the eyes, don't look around the
eyes, look into my eyes, you're under.


In [12]:
print(textwrap.fill(s, 40, subsequent_indent="    |"))

Look into my eyes, look into my eyes,
    |the eyes, the eyes, the eyes, not
    |around the eyes, don't look around
    |the eyes, look into my eyes, you're
    |under.


In [29]:
textwrap.wrap(s,width=25)

['Look into my eyes, look',
 'into my eyes, the eyes,',
 'the eyes, the eyes, not',
 "around the eyes, don't",
 'look around the eyes,',
 "look into my eyes, you're",
 'under.']

In [30]:
text="afdsafsafsakfhsajkgbakghfsakfgs asdfdajhfgjagsdf asfhajhf asdfksfg rgtreoteg bddsh"

In [31]:
textwrap.wrap(text,width=10)

['afdsafsafs',
 'akfhsajkgb',
 'akghfsakfg',
 's asdfdajh',
 'fgjagsdf',
 'asfhajhf',
 'asdfksfg',
 'rgtreoteg',
 'bddsh']

## Discussion

El modulo textwrap es una forma sencilla de limpiar el texto para imprimirlo, especialmente si desea que la salida se ajuste bien en la terminal. En cuanto al tamaño de la terminal, puede obtenerlo usando os.get_terminal_size(). 

Por ejemplo:



In [4]:

import os

os.get_terminal_size().columns


OSError: [Errno 25] Inappropriate ioctl for device

El error OSError: [Errno 25] Inappropriate ioctl for device.  

Ocurre porque `os.get_terminal_size()` está siendo llamado en un entorno donde no hay un terminal asociado, como en un Jupyter Notebook.

Para evitar este error, puedes usar una comprobación para asegurarte de que el código solo se ejecute si hay un terminal disponible. Aquí tienes una forma de hacerlo:

In [7]:
import os
import shutil

def get_terminal_width():
    try:
        return os.get_terminal_size().columns
    except OSError as e:
        print(e)
        # Fallback to a default value or use shutil.get_terminal_size()
        # Si la fallback falla, usar shutil.get_terminal_size()
        return shutil.get_terminal_size().columns

terminal_width = get_terminal_width()
print(f"Ancho del terminal: {terminal_width}")

[Errno 25] Inappropriate ioctl for device
Ancho del terminal: 80


# Manejo de entidades HTML y XML en texto

2.17

- Problema
        
        Desea reemplazar entidades HTML o XML como & entity; o & # code; con su texto correspondiente. Alternativamente, necesita producir texto, pero escapar de ciertos caracteres (por ejemplo, <,> o &).


- Solución

        Si está produciendo texto, reemplazar caracteres especiales como <o> es relativamente fácil si utiliza la función html.escape().   
        
Por ejemplo:

In [11]:
s = 'Elements are written as "<tag>text</tag>".'
s

'Elements are written as "<tag>text</tag>".'

In [12]:
import html

In [13]:
print(html.escape(s))

Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


In [14]:
print(html.escape(s, quote=False))

Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


In [15]:
t = 'The prompt is &gt;&gt;&gt;'

In [63]:
html.unescape(t)

'The prompt is >>>'

In [65]:
s = 'Spicy &quot;Jalape&#241;o&quot.'
html.unescape(s)

'Spicy "Jalapeño".'

In [2]:
import xml.etree.ElementTree as ET

xml_data = '''<root>
    <child1>data1</child1>
    <child2>data2</child2>
    <child3>data3</child3>
</root>'''

# Parse the XML data
root = ET.fromstring(xml_data)

# Function to tokenize XML
def tokenize_xml(element):
    tokens = []
    for child in element:
        tokens.append((child.tag, child.text))
        tokens.extend(tokenize_xml(child))
    return tokens

# Tokenize the XML data
tokens = tokenize_xml(root)
print(tokens)

[('child1', 'data1'), ('child2', 'data2'), ('child3', 'data3')]


# Tokenización de texto

2.18

- Problema
        
        Tiene una cadena que desea analizar de izquierda a derecha en una secuencia de tokens.  


- Solución
        
        Suponga que tiene una cadena de texto como esta:

In [87]:
text = 'foo=23+42*10.5/4%2 // 34**2-3' 

Para tokenizar la cadena, necesita hacer más que simplemente hacer coincidir patrones.  
Tambien necesitas alguna forma de identificar el tipo de patrón.   

Por ejemplo, es posible que desee convierte la cadena en una secuencia de pares como esta:

```python
tokens = [('Variable', 'foo'), ('igual','='), ('numero', '23'), ('suma','+'),('numero', '42'), ('multipacion', '*'), ('numero', 10')]
```
Para hacer este tipo de división, el primer paso es definir todos los tokens posibles, incluidos espacios en blanco, mediante patrones de expresión regular utilizando grupos de captura con nombre como este:

In [16]:
import re
p1 = r'(?P<Variable>[a-zA-Z_][a-zA-Z_0-9]*)' 
p2 = r'(?P<igual>=)'
p3 = r'(?P<suma>\+)'
p4 = r'(?P<mulpiplicacion>\*)'
p5 = r'(?P<resta>-)'
p6 = r'(?P<divicion>\/)\s'
p7 = r'(?P<modulo>%)'
p8 = r'(?P<divicion_exacta>\//)'
p9 = r'(?P<float>(\d+[.]\d+))\s*'
p0 = r'(?P<int>(\d+\s*$|\d+\s*))'
p  = r'(?P<espacio>\s+)'
pp = r'(?P<potencia>\*\*)'
LT = r'(?P<Menor><)'
LE = r'(?P<Menor_Igual><=)'
GT = r'(?P<Mayor>>)'
GE = r'(?P<Mayor_Igual>>=)'
LP = r'(?P<Parentecis_izq>\()'
RP = r'(?P<Parentecis_der>\))'
master_pat = re.compile('|'.join([LE,GE,LT,GT,pp,p1,p2,p3,p4,p5,p6,p7,p8,p9,p0,p,LP,RP]))

In [17]:
scanner = master_pat.scanner('foo = 42')

In [18]:
a=scanner.search()

In [19]:
a.groupdict()

{'Menor_Igual': None,
 'Mayor_Igual': None,
 'Menor': None,
 'Mayor': None,
 'potencia': None,
 'Variable': 'foo',
 'igual': None,
 'suma': None,
 'mulpiplicacion': None,
 'resta': None,
 'divicion': None,
 'modulo': None,
 'divicion_exacta': None,
 'float': None,
 'int': None,
 'espacio': None,
 'Parentecis_izq': None,
 'Parentecis_der': None}

In [245]:
bus=master_pat.scanner(text)

In [246]:
[(i.lastgroup,i.group().strip()) for i in iter(bus.search,None)]

[('Variable', 'foo'),
 ('espacio', ''),
 ('igual', '='),
 ('espacio', ''),
 ('int', '23'),
 ('suma', '+'),
 ('int', '42'),
 ('mulpiplicacion', '*'),
 ('float', '10.5'),
 ('int', '4'),
 ('modulo', '%'),
 ('int', '2'),
 ('divicion_exacta', '//'),
 ('espacio', ''),
 ('int', '34'),
 ('potencia', '**'),
 ('int', '2'),
 ('resta', '-'),
 ('int', '3'),
 ('Mayor', '>'),
 ('espacio', ''),
 ('int', '7'),
 ('Menor', '<'),
 ('espacio', ''),
 ('int', '76'),
 ('Mayor_Igual', '>='),
 ('int', '13'),
 ('Menor_Igual', '<='),
 ('int', '14'),
 ('Parentecis_izq', '('),
 ('Variable', 'hola'),
 ('Parentecis_der', ')'),
 ('Parentecis_izq', '('),
 ('float', '12.90'),
 ('mulpiplicacion', '*'),
 ('int', '2'),
 ('Parentecis_der', ')')]

Para tomar esta técnica y ponerla en código, se puede limpiar y empaquetar fácilmente
en un generador como este:

In [20]:
from collections import namedtuple

Token = namedtuple('Token', ['type','value'])

def generate_tokens(pat, text):
    scanner = pat.scanner(text)
    for m in iter(scanner.search, None):
        yield Token(m.lastgroup, m.group().strip())

In [21]:
text = 'foo = 23+42*10.5/4%2 // 34**2-3 > 7 < 76 >=13<=14 (hola)(12.90*2)' 

for tok in generate_tokens(master_pat, text):
    print(tok)

Token(type='Variable', value='foo')
Token(type='espacio', value='')
Token(type='igual', value='=')
Token(type='espacio', value='')
Token(type='int', value='23')
Token(type='suma', value='+')
Token(type='int', value='42')
Token(type='mulpiplicacion', value='*')
Token(type='float', value='10.5')
Token(type='int', value='4')
Token(type='modulo', value='%')
Token(type='int', value='2')
Token(type='divicion_exacta', value='//')
Token(type='espacio', value='')
Token(type='int', value='34')
Token(type='potencia', value='**')
Token(type='int', value='2')
Token(type='resta', value='-')
Token(type='int', value='3')
Token(type='Mayor', value='>')
Token(type='espacio', value='')
Token(type='int', value='7')
Token(type='Menor', value='<')
Token(type='espacio', value='')
Token(type='int', value='76')
Token(type='Mayor_Igual', value='>=')
Token(type='int', value='13')
Token(type='Menor_Igual', value='<=')
Token(type='int', value='14')
Token(type='Parentecis_izq', value='(')
Token(type='Variable', val

Si desea filtrar el flujo de tokens de alguna manera, puede definir más generador
funciones o use una expresión generadora. Por ejemplo, así es como puede filtrar
todos los tokens de espacios en blanco.

In [22]:
tokens = (tok for tok in generate_tokens(master_pat, text) if tok.type != 'espacio')
#for tok in tokens:
#    print(tok)

In [23]:
list(tokens)

[Token(type='Variable', value='foo'),
 Token(type='igual', value='='),
 Token(type='int', value='23'),
 Token(type='suma', value='+'),
 Token(type='int', value='42'),
 Token(type='mulpiplicacion', value='*'),
 Token(type='float', value='10.5'),
 Token(type='int', value='4'),
 Token(type='modulo', value='%'),
 Token(type='int', value='2'),
 Token(type='divicion_exacta', value='//'),
 Token(type='int', value='34'),
 Token(type='potencia', value='**'),
 Token(type='int', value='2'),
 Token(type='resta', value='-'),
 Token(type='int', value='3'),
 Token(type='Mayor', value='>'),
 Token(type='int', value='7'),
 Token(type='Menor', value='<'),
 Token(type='int', value='76'),
 Token(type='Mayor_Igual', value='>='),
 Token(type='int', value='13'),
 Token(type='Menor_Igual', value='<='),
 Token(type='int', value='14'),
 Token(type='Parentecis_izq', value='('),
 Token(type='Variable', value='hola'),
 Token(type='Parentecis_der', value=')'),
 Token(type='Parentecis_izq', value='('),
 Token(type='

La tokenización es a menudo el primer paso para tipos más avanzados de análisis y manejo de texto.  
Para utilizar la técnica de escaneo que se muestra, hay algunos detalles importantes a tener en cuenta.  
Primero, debe asegurarse de identificar todas las secuencias de texto posibles que aparecen en la entrada con un patrón de respuesta correspondiente. Si se encuentra algún texto que no coincida, el escaneo simplemente se detiene. Es por eso que fue necesario especificar el token de espacios en blanco (WS) en el ejemplo.  

El orden de los tokens en la expresión regular maestra también es importante. Cuando coincida, vuelva intenta hacer coincidir los patrones en el orden especificado. Por lo tanto, si un patrón es una subcadena de un patrón más largo, debe asegurarse de que el patrón más largo vaya primero. Por ejemplo:

# Writing a Simple Recursive Descent Parser
# Escribir un analizador de descenso recursivo simple

- Problema
        
        You need to parse text according to a set of grammar rules and perform actions or build an abstract syntax tree representing the input. The grammar is small, so you’d prefer to just write the parser yourself as opposed to using some kind of framework.

        Necesita analizar el texto de acuerdo con un conjunto de reglas de gramática y realizar acciones o construir un árbol de sintaxis abstracta que represente la entrada. La gramática es pequeña, por lo que preferiría escribir el analizador usted mismo en lugar de usar algún tipo de marco.

- Solución:

        En este problema, nos centramos en el problema de analizar texto de acuerdo con una gramática particular. Para hacer esto, probablemente debería comenzar teniendo una especificación formal de la gramática en forma de BNF o EBNF. Por ejemplo, una gramática para expresiones aritméticas simples podría verse así:


```text
        xpr    ::= expr + term
               |
               expr - term
               |
               term
        term   ::= term * factor
               |
                term / factor
               |
                factor
        factor ::= ( expr )
               | 
                NUM
```

O, altetnativamente, en forma de EBNF:

```text
        expr ::= term { (+|-) term }*
        term ::= factor { (*|/) factor }*
        factor ::= ( expr )
                | NUM
```
  

En `EBNF`, las partes de una regla encerradas en `{ ... }*` son opcionales. El `*` significa cero o más repeticiones (el mismo significado que en una expresión regular).  

Ahora, si no está familiarizado con los mecanismos de trabajo con un BNF, piense en él como una especificación de reglas de sustitución o reemplazo donde los símbolos en el lado izquierdo pueden ser reemplazados por los símbolos en el lado derecho (o viceversa).   

Generalmente, lo que sucede durante el análisis es que intenta hacer coincidir el texto de entrada con la gramática haciendo varias sustituciones y expansiones utilizando el BNF.   

Para ilustrar, suponga que está analizando una expresión como `3 + 4 * 5` . Esta expresión primero debería descomponerse en una secuencia de tokens, utilizando las técnicas descritas en la Receta 2.18.
El resultado podría ser una secuencia de tokens como esta:

```text
NUM + NUM * NUM
```



A partir de ahí, el análisis implica intentar hacer coincidir la gramática con los tokens de entrada haciendo sustituciones:  

```text
expr
expr ::= term { (+|-) term }*
expr ::= factor { (*|/) factor }* { (+|-) term }*
expr ::= NUM { (*|/) factor }* { (+|-) term }*
expr ::= NUM { (+|-) term }*
expr ::= NUM + term { (+|-) term }*
expr ::= NUM + factor { (*|/) factor }* { (+|-) term }*
expr ::= NUM + NUM { (*|/) factor}* { (+|-) term }*
expr ::= NUM + NUM * factor { (*|/) factor }* { (+|-) term }*
expr ::= NUM + NUM * NUM { (*|/) factor }* { (+|-) term }*
expr ::= NUM + NUM * NUM { (+|-) term }*
expr ::= NUM + NUM * NUM
```

Siguientes todos los pasos de sustitución requiere un poco de café, pero están impulsados por mirar la entrada e intentar hacer coincidirla con las reglas de gramática. El primer token de entrada es un NUM, por lo que las sustituciones se centran primero en hacer coincidir esa parte. Una vez que se ha hecho coincidir, la atención se traslada al siguiente token de + y así sucesivamente.   
Ciertas partes del lado derecho (por ejemplo, `{ (*/) factor }*)` desaparecen cuando se determina que no pueden hacer coincidir el siguiente token. En un análisis exitoso, todo el lado derecho se expande completamente para hacer coincidir la secuencia de tokens de entrada.

Con todo el trasfondo anterior en su lugar, aquí hay una receta simple que muestra cómo construir un evaluador de expresiones de descenso recursivo:

In [6]:
import re
import collections

NUM    = r'(?P<NUM>\d+)'
PLUS   = r'(?P<PLUS>\+)'
MINUS  = r'(?P<MINUS>-)'
TIMES  = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS     = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES,
DIVIDE, LPAREN, RPAREN, WS]))

# Tokenizer
Token = collections.namedtuple('Token', ['type','value'])

def generate_tokens(text):
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type != 'WS':
            yield tok

In [7]:
# Parser
class ExpressionEvaluator:
    '''
    Implentacion de un parser descendente recursivo.

    Cada metodo
        implementa una sola regla de la gramática. Use el método ._accept()
        para comprobar y aceptar el token de lookahead actual. Use el método
        ._expect() para exactamente coincidir y descartar el siguiente token
    '''
    def parse(self,text):
        self.tokens = generate_tokens(text)
        self.tok = None
        # Last symbol consumed (ultimo simbolo consumido)
        self.nexttok = None
        # Next symbol tokenized (proximo token)
        self._advance()
        # Load first lookahead token ()
        return self.expr()
    
    def _advance(self):
        'Advance one token ahead'
        self.tok, self.nexttok = self.nexttok, next(self.tokens, None)

    def _accept(self,toktype):
        'Test and consume the next token if it matches toktype'
        if self.nexttok and self.nexttok.type == toktype:
            self._advance()
            return True
        else:
            return False
    
    def _expect(self,toktype):
        'Consume next token if it matches toktype or raise SyntaxError'
        if not self._accept(toktype):
            raise SyntaxError('Expected ' + toktype)
    
    # Grammar rules follow
    def expr(self):
        "expression ::= term { ('+'|'-') term }*"
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval += right
            elif op == 'MINUS':
                exprval -= right
        return exprval

    def term(self):
        "term ::= factor { ('*'|'/') factor }*"
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval *= right
            elif op == 'DIVIDE':
                termval /= right
        return termval

    def factor(self):
        "factor ::= NUM | ( expr )"
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')
        


In [8]:
e = ExpressionEvaluator()


In [9]:
e.parse('2')

2

In [10]:
e.parse('2 + 3')

5

In [11]:
e.parse('2 + 3 * 4')

14

In [12]:
e.parse('2 + (3 + 4) * 5')

37

In [13]:
try:
    e.parse('2 + (3 + * 4)')
except Exception as error:
    print(type(error), error)

<class 'SyntaxError'> Expected NUMBER or LPAREN


In [14]:
e.parse('2 + (3 + 4 * 5)')

25

# Realización de operaciones de texto en cadenas de bytes

2.20

- Problema

        Desea realizar operaciones de texto comunes (por ejemplo, eliminar, buscar y reemplazar en cadenas de bytes.  
        
- Solución

        Las cadenas de bytes ya admiten la mayoría de las mismas operaciones integradas que las cadenas de texto.   

Por ejemplo:

In [248]:
data = b'Hello World'
data[0:5]

b'Hello'

In [249]:
data.startswith(b'Hello')

True

In [250]:
data.split()

[b'Hello', b'World']

In [251]:
data.replace(b'Hello', b'Hello Cruel')

b'Hello Cruel World'

In [252]:
data = bytearray(b'Hello World')
data[0:5]

bytearray(b'Hello')

In [253]:
data.startswith(b'Hello')

True

In [254]:
data.split()

[bytearray(b'Hello'), bytearray(b'World')]

In [255]:
data.replace(b'Hello', b'Hello Cruel')

bytearray(b'Hello Cruel World')

In [24]:
data = b'FOO:BAR,SPAM'

import re

re.split('[:,]',data)

TypeError: cannot use a string pattern on a bytes-like object

In [25]:
data = b'FOO:BAR,SPAM'
import re
re.split(b'[:,]',data)

[b'FOO', b'BAR', b'SPAM']

In [26]:
a = 'Hello World'
b = b'Hello World'
a[0],b[0]

('H', 72)